# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.<p>
The variables in the files are:<p>
code_postal: postal code of the insured property<p>
nb_pieces: number of rooms<p>
veranda: presence of a veranda<p>
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)<p>
sit_fam: family situation of the insured<p>
profession: profession of the insured<p>
nature_lri: nature of the insured property<p>
societe: company covering the risk<p>
cdomact: area of activity of the insured<p>
niv_urbanisation: level of urbanization in which is the insured property<p>
age_societaire: age of the member<p>
objets_precieux: presence of precious objects in the insured property<p>
pres_contrat_auto: holding a contract of auto insurance<p>
formule: guarantee formula<p>
sit_juridique: legal situation<p>
surf_habitat: area of insured property<p>
patrimoine_mob: amount of heritage insured furniture<p>
var1 - ... - var11: non-explicit variables<p>


## Wikipedia page views (SMAPE metric)

In [52]:
import pandas as pd
import numpy as np

In [53]:
### This is data from wikipedia3 archive in data folder

train = pd.read_csv("wikipedia_train3.csv")
test = pd.read_csv("wikipedia_test3.csv")

In [27]:
train.shape

(2882860, 3)

In [28]:
train.dropna().shape

(2882860, 3)

In [29]:
train.head()

,Page,date,Visits
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-01-01,2660.0
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-01-01,9936.0
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-01-01,1712.0
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-01-01,3175.0
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-01-01,1682.0


In [30]:
train.tail()

,Page,date,Visits
2882855,The_Revenant_–_Der_Rückkehrer_de.wikipedia.org...,2016-08-31,942.0
2882856,Especial:Entrar_es.wikipedia.org_all-access_sp...,2016-08-31,1193.0
2882857,Especial:Libro_es.wikipedia.org_all-access_spider,2016-08-31,1862.0
2882858,Special:Search_es.wikipedia.org_all-access_spider,2016-08-31,7217.0
2882859,Wikipedia:Portada_es.wikipedia.org_all-access_...,2016-08-31,57752.0


In [31]:
train.Page.value_counts().shape

(11815,)

In [32]:
train.date.value_counts().shape

(244,)

In [33]:
test.shape

(732530, 3)

In [34]:
test.dropna().shape

(732530, 3)

In [35]:
test.head()

,Page,date,Visits
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-09-10,640.0
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-09-10,5609.0
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-09-10,3121.0
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-09-10,362.0
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-09-10,533.0


In [36]:
test.tail()

,Page,date,Visits
732525,The_Revenant_–_Der_Rückkehrer_de.wikipedia.org...,2016-11-10,1836.0
732526,Especial:Entrar_es.wikipedia.org_all-access_sp...,2016-11-10,568.0
732527,Especial:Libro_es.wikipedia.org_all-access_spider,2016-11-10,1561.0
732528,Special:Search_es.wikipedia.org_all-access_spider,2016-11-10,4624.0
732529,Wikipedia:Portada_es.wikipedia.org_all-access_...,2016-11-10,56413.0


In [13]:
test.Page.value_counts().shape

(11815,)

In [14]:
test.date.value_counts().shape

(62,)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [54]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [57]:
traintrain = train.loc[:11815*188-1, :]
trainvalidation = train.loc[11815*196:, :]

In [58]:
traintrain_pivot = traintrain.pivot_table(values='Visits', index='Page', columns='date')
trainvalidation_pivot = trainvalidation.pivot_table(values='Visits', index='Page', columns='date')

In [59]:
trainvalidation_pivot_pred = pd.DataFrame({i:np.median(traintrain_pivot.iloc[:,((-1)*15):], axis = 1) for i in trainvalidation_pivot.columns[0:]})

trainvalidation_pivot_pred['Page'] = trainvalidation_pivot.index
trainvalidation_pivot_predm = pd.melt(trainvalidation_pivot_pred, value_vars=trainvalidation_pivot_pred.columns[:-1], id_vars='Page')

trainvalidation_pivotm = pd.melt(trainvalidation_pivot, value_vars=trainvalidation_pivot.columns[1:], id_vars='Page')
trainvalidation_pivotm['pred_Visits'] = trainvalidation_pivot_predm.value
trainvalidation_pivotm.rename(columns={'variable':'date', 'value':'Visits'}, inplace = True)

smape_fast(trainvalidation_pivotm['Visits'], trainvalidation_pivotm['pred_Visits'])

42.124723792105939

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [75]:
train = pd.read_csv("wikipedia_train3.csv", parse_dates=['date'])
test = pd.read_csv("wikipedia_test3.csv", parse_dates=['date'])

In [76]:
train['Month']     = train["date"].dt.month
train['Day']       = train["date"].dt.day
train['DayOfWeek'] = train["date"].dt.dayofweek

test['Month']     = test["date"].dt.month
test['Day']       = test["date"].dt.day
test['DayOfWeek'] = test["date"].dt.dayofweek

In [77]:
from dateutil.relativedelta import relativedelta
import math

In [78]:
### Your code here
def create_validation(df, start_date):
    return df.loc[(df['date'] >= pd.to_datetime(start_date) - relativedelta(days=0)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=6))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(months=6)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=12))].index

In [79]:
train_dates = ['2014-06-01', '2014-03-01', '2015-05-01', '2015-07-01']

myCViterator = []
for i in train_dates:
    trainIndices, valIndices = create_validation(train, i)
    myCViterator.append((trainIndices, valIndices))

In [69]:
train.head()

,Page,date,Visits,Month,Day,DayOfWeek
0,Special:Search_zh.wikipedia.org_all-access_spider,2016-01-01,2660.0,1,1,4
1,Wikipedia:首页_zh.wikipedia.org_all-access_spider,2016-01-01,9936.0,1,1,4
2,Project:上传_zh.wikipedia.org_all-access_spider,2016-01-01,1712.0,1,1,4
3,Special:用户登录_zh.wikipedia.org_all-access_spider,2016-01-01,3175.0,1,1,4
4,Championnat_d'Europe_de_football_2016_fr.wikip...,2016-01-01,1682.0,1,1,4


In [80]:
X = train.drop(['Page', 'date', 'Visits'], axis = 1)
y = train['Visits']

X_test = test.drop(['Page', 'date', 'Visits'], axis = 1)
y_test = test['Visits']

In [81]:
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


regressor = SVR()

pipeline_r = Pipeline([('regressor', regressor)])
param_grid = [
  {'regressor__C': [0.01, 0.1, 1, 10, 100, 1000], 'regressor__kernel': ['linear']},
  {'regressor__C': [0.01, 0.1, 1, 10, 100, 1000], 'regressor__gamma': [0.001, 0.0001], 'regressor__kernel': ['rbf']},
 ]
grid_search = GridSearchCV(pipeline_r, param_grid=param_grid, cv=myCViterator)

%time grid_search.fit(X,y)

ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required.

## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [82]:
train = pd.read_csv('../data/train_french.csv')
test = pd.read_csv('../data/test_french.csv')

In [88]:
train.shape, test.shape

((200000, 29), (75968, 29))

In [83]:
train.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,62510,1,NON,500,Indivividu,agent; employe,Sans information,2,34,A,...,0,194,49,0,0,1,NON,30,2,67.68
1,83126,2,NON,500,Indivividu,cadre; ingenieur,Sans information,2,37,A,...,0,0,100,0,0,5,NON,12,1,120.60
2,68224,3,NON,500,Indivividu,enseignant; formateur; chercheur,Maison,1,1,E,...,1,98,100,0,0,5,NON,30,2,100.62
3,30263,3,NON,500,Famille,agent; employe,Maison,2,34,A,...,0,151,38,0,0,1,NON,15,1,58.73
4,72231,2,NON,500,Famille,educateur; animateur; moniteur,Appartement,1,19,D,...,0,39,100,0,0,1,NON,0,2,132.65


In [95]:
train.nature_lri.value_counts()

Appartement                        110302
Maison                              68488
Sans information                    19262
R�sidence de retraite                1145
Mobil home                            799
Local utilitaire distinct (LUD)         4
Name: nature_lri, dtype: int64

In [91]:
train.profession.value_counts()

enseignant; formateur; chercheur                             72666
agent; employe                                               54004
cadre; ingenieur                                             23711
educateur; animateur; moniteur                               10068
technicien                                                    9718
demandeur d'emploi                                            4859
homme ou femme au foyer                                       4094
profession liberale                                           3997
etudiant                                                      3592
profession de l'information des arts et des spectacles        3497
ouvrier                                                       3139
profession intermediaire de la sante et du travail social     2254
commercant                                                    1205
contremaitre; agent de maitrise                               1023
chef d'entreprise                                             

In [85]:
test.head()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
0,6029,5,OUI,500,Famille,agent; employe,Maison,1,7,C,...,0,164,41,0,0,1,NON,30,1,107.94
1,62128,2,NON,0,Famille,enseignant; formateur; chercheur,Appartement,1,1,D,...,1,85,100,0,0,1,NON,30,2,80.41
2,81065,4,NON,1000,Indivividu,homme ou femme au foyer,Sans information,2,99,~,...,0,94,100,0,0,1,NON,30,2,142.12
3,5023,3,NON,500,Indivividu,agent; employe,Maison,2,38,C,...,0,101,100,0,0,5,NON,30,2,76.55
4,62178,2,NON,500,Famille,enseignant; formateur; chercheur,Appartement,1,1,~,...,1,125,100,1,0,1,NON,0,2,150.45


In [84]:
train.tail()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
199995,50129,3,NON,0,Famille,technicien,Sans information,2,0,C,...,0,77,100,0,0,1,NON,12,2,144.54
199996,15014,3,NON,500,Indivividu,cadre; ingenieur,Appartement,2,41,A,...,0,112,100,0,0,1,NON,11,2,76.29
199997,94017,3,NON,1000,Famille,enseignant; formateur; chercheur,Maison,1,2,B,...,1,110,100,1,0,1,NON,30,1,156.20
199998,83924,2,NON,500,Indivividu,enseignant; formateur; chercheur,Appartement,1,1,A,...,0,99,100,1,0,1,NON,30,2,63.35
199999,62427,1,NON,1000,Famille,profession liberale,Appartement,2,41,A,...,0,79,100,0,0,1,NON,30,1,146.68


In [87]:
test.tail()

,code_postal,nb_pieces,veranda,biens_nomades,sit_fam,profession,nature_lri,societe,domaine_activite,niv_urbanisation,...,var3,var4,var5,var6,var7,var8,var9,var10,var11,prime_tot_ttc
75963,91215,6,OUI,1000,Famille,enseignant; formateur; chercheur,Maison,1,2,C,...,1,156,39,1,1,1,NON,30,1,128.59
75964,58083,6,OUI,1000,Famille,enseignant; formateur; chercheur,Maison,1,1,E,...,1,27,100,0,1,1,OUI,30,1,219.17
75965,7264,6,OUI,1000,Famille,enseignant; formateur; chercheur,Maison,1,2,C,...,0,135,34,1,0,1,NON,11,1,106.99
75966,50244,6,OUI,500,Famille,enseignant; formateur; chercheur,Maison,1,1,F,...,0,96,100,0,0,1,OUI,0,1,87.22
75967,75111,6,OUI,1000,Indivividu,enseignant; formateur; chercheur,Sans information,1,2,B,...,0,143,36,0,0,1,NON,30,1,89.63


In [104]:
train.drop('code_postal', inplace = True, axis=1)
test.drop('code_postal', inplace = True, axis=1)

In [105]:
y_train = train.prime_tot_ttc
train.drop('prime_tot_ttc', inplace = True, axis=1)

y_test = test.prime_tot_ttc
test.drop('prime_tot_ttc', inplace = True, axis=1)

In [101]:
for col in train.columns[1:]:
    print(col, len(set(train[col])))

nb_pieces 5
veranda 2
biens_nomades 5
sit_fam 2
profession 18
nature_lri 6
societe 2
domaine_activite 44
niv_urbanisation 8
age_societaire 83
objets_precieux 2
pres_contrat_auto 2
formule 2
sit_juridique 4
surf_habitat 203
patrimoine_mob 46247
var1 40
var2 10
var3 2
var4 297
var5 22
var6 2
var7 2
var8 6
var9 2
var10 15
var11 2
prime_tot_ttc 9552


In [109]:
numerical_col = ['var4', 'var1', 'patrimoine_mob', 'surf_habitat', 'age_societaire', 'domaine_activite']
binary_col = ['veranda', 'sit_fam', 'societe','objets_precieux','pres_contrat_auto',
             'formule', 'var3','var6','var7','var9','var11']
categorical_col = list(set(train.columns).difference(set(numerical_col).union(set(binary_col))))

In [116]:
# To be used in training, data must be properly encoded
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
# function to encode categorical data


def __encode_binary(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat([ df[cat_cols] for df in df_list],axis=0).fillna('').apply(lambda x: d[x.name].fit(x))
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(lambda x: d[x.name].transform(x))



In [114]:
__encode_binary(df_list=[train, test], cat_cols=binary_col)

In [119]:
from sklearn.feature_extraction import DictVectorizer as DV

encoder = DV(sparse = False)
codes = pd.concat([train[categorical_col], test[categorical_col]],axis=0)

encoder.fit(codes.T.to_dict().values())

train_c = encoder.transform(train[categorical_col])
test_c = encoder.transform(test[categorical_col])

/home/marharyta/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:881: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  "columns will be omitted.", UserWarning)


AttributeError: 'str' object has no attribute 'items'